## Export wflow staticmaps netcdf to raster files

In order to inspect or (manually) modify wflow staticmaps it is convenient to export the maps to a raster format. Here we show how to read the model maps and save to a so-called mapstack (i.e.: a set of raster files with identical grid) using hydromt.  

### Load dependencies

In [ ]:
import pandas as pd
import xarray as xr
import numpy as  np
from os.path import join, dirname
import os
import hydromt

### Read wflow staticmaps

hydroMT provides an easy method to read the model schematization trought the Model API.

In [ ]:
root = 'wflow_piave_subbasin' 
mod = hydromt.WflowModel(root, mode='r')
ds = mod.staticmaps  # here the staticmaps netcdf is loaded 
print(ds)

### Write netcdf to mapstack

The raster module provides many raster GIS methods throught the **raster** Dataset accessor. To write a Dataset to a mapstack one line with code is sufficient. We only need to provide the output folder in which all raster files are saved. The default output format is *GeoTIFF*, but this can be changed with the `driver` argument. To write to PCRaster map-files it is recommended to have PCRaster python installed.

In [ ]:
outdir =join(root, 'staticmaps')
ds.raster.to_mapstack(outdir)

Now the model files can easily be inspected and modified e.g. QGIS.

NOTE: in QGIS, you can also visualize but direct modification is not (yet) possible.

### Create staticmaps netcdf files based on mapstack

If you want to update the staticmaps after modification the maps can be read into a Dataset by hydromt. We recommend the following workflow:

* read the original model
* read the updated mapstack
* change the model root to write the updated model to a new directory
* update the staticmaps of the model
* write the model

NOTE: We do not read the forcing as it is probably faster to just copy the file instead of loading it into python and writing it back to netcdf.

NOTE: The staticgeoms might be changed because of manual changes in the wflow_river, lakes, reservoir or glacier staticmaps and are therefore not read here. To change these maps we recommend using the hydromt update method to keep the staticgeoms and maps aligned.

In [ ]:
# read the original model
root = 'wflow_piave_subbasin' 
mod = hydromt.WflowModel(root, mode='r')
mod.read_staticmaps()
mod.read_config()

In [ ]:
# read the updated mapstack
# NOTE: The mapstack does not have to include all staticmaps, only the once that are found will be updated.
# The name of the staticmap should however have to be unchanged. 
ds_updated = hydromt.open_mfraster(join(root, 'staticmaps', '*.tif'))

In [ ]:
# change root to a new directory
root_updated = 'wflow_piave_subbasin_updated'
mod.set_root(root_updated, mode='w')  

In [ ]:
# update the model staticmaps
mod.set_staticmaps(ds_updated)

In [ ]:
# write the model to the new directory
mod.write()